In [1]:
import os 
%pwd


'd:\\My Workspace\\proj_file\\End-To-End-Wine-Quality-Pred\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\My Workspace\\proj_file\\End-To-End-Wine-Quality-Pred'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/viyas52/End-To-End-Wine-Quality-Pred.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="viyas52"
os.environ["MLFLOW_TRACKING_PASSWORD"]="396092e9fb18dff075743b0080074533456e33c1"


In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [11]:
from src.constants import *
from src.utils.common import read_yaml, create_directories, save_json


In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.TUNING
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/viyas52/End-To-End-Wine-Quality-Pred.mlflow",
           
        )

        return model_evaluation_config
        
    

In [13]:

import os
import pandas as pd
from sklearn.metrics import accuracy_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [14]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        score = accuracy_score(actual,pred)
        return score
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            acc_score = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"accuracy score": acc_score}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy_score", acc_score)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestClassifier")
            else:
                mlflow.sklearn.log_model(model, "model")

In [15]:
from src.exception import CustomException
import sys

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise CustomException(e,sys)

[ 2024-06-22 18:45:34,911 ] 33 root - INFO - yaml file: config\config.yaml loaded successfully
[ 2024-06-22 18:45:34,913 ] 33 root - INFO - yaml file: params.yaml loaded successfully
[ 2024-06-22 18:45:34,916 ] 33 root - INFO - yaml file: schema.yaml loaded successfully
[ 2024-06-22 18:45:34,917 ] 53 root - INFO - created directory at: artifacts
[ 2024-06-22 18:45:34,918 ] 53 root - INFO - created directory at: artifacts/model_evaluation
[ 2024-06-22 18:45:35,402 ] 67 root - INFO - json file saved at: artifacts\model_evaluation\metrics.json


Registered model 'RandomForestClassifier' already exists. Creating a new version of this model...
2024/06/22 18:45:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: RandomForestClassifier, version 4
Created version '4' of model 'RandomForestClassifier'.
